In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [2]:
data_sample=pd.read_csv("individual_book_train\stock_0.csv")
data_sample

FileNotFoundError: [Errno 2] No such file or directory: 'individual_book_train\\stock_0.csv'

In [3]:
data_sample['interval_id'] = data_sample['seconds_in_bucket'] // 60

# Create a new 'id' column that combines the 'time_id' and 'interval_id' to create a unique ID
data_sample['minute_id'] = data_sample['time_id'].astype(str) + '-' + data_sample['interval_id'].astype(str)

# You can now drop 'interval_id' if you want
data_sample = data_sample.drop('interval_id', axis=1)
data_sample

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2,stock_id,minute_id
0,5,0,1.001422,1.002301,1.001370,1.002353,3,226,2,100,0,5-0
1,5,1,1.001422,1.002301,1.001370,1.002353,3,100,2,100,0,5-0
2,5,5,1.001422,1.002301,1.001370,1.002405,3,100,2,100,0,5-0
3,5,6,1.001422,1.002301,1.001370,1.002405,3,126,2,100,0,5-0
4,5,7,1.001422,1.002301,1.001370,1.002405,3,126,2,100,0,5-0
...,...,...,...,...,...,...,...,...,...,...,...,...
917548,32767,568,0.998275,0.998754,0.997796,0.998946,90,90,48,28,0,32767-9
917549,32767,569,0.998275,0.998754,0.997892,0.998946,91,90,200,28,0,32767-9
917550,32767,571,0.998275,0.998754,0.997892,0.998946,91,90,100,28,0,32767-9
917551,32767,572,0.998275,0.998754,0.997892,0.998946,92,90,100,28,0,32767-9


In [4]:
def calc_wap(df):
    wap = (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1'])/(df['bid_size1'] + df['ask_size1'])
    return wap
def calc_wap2(df):
    wap = (df['bid_price2'] * df['ask_size2'] + df['ask_price2'] * df['bid_size2'])/(df['bid_size2'] + df['ask_size2'])
    return wap
def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff()
def realized_volatility(series):
    return np.sqrt(np.sum(series**2))

In [32]:
def preprocessor_book(stock_id):
    #data reading
    dir_path=f"individual_book_train/stock_{stock_id}.csv"
    data=pd.read_csv(dir_path)
    data['interval_id'] = data['seconds_in_bucket'] // 60
    
    data['minute_id'] = data['time_id'].astype(str) + '-' + data['interval_id'].astype(str)
    data = data.drop('interval_id', axis=1)
    #calculate return etc
    data['wap1'] =(data['bid_price1'] * data['ask_size1'] + data['ask_price1'] * data['bid_size1'])/(data['bid_size1'] + data['ask_size1'])
    data['log_return'] = data.groupby('time_id')['wap1'].transform(log_return)
    data['wap2'] = calc_wap2(data)
    data['log_return2'] = data.groupby('time_id')['wap2'].transform(log_return)
    
    data['wap_balance'] = abs(data['wap1'] - data['wap2'])
    
    data['price_spread'] = (data['ask_price1'] - data['bid_price1']) / ((data['ask_price1'] + data['bid_price1'])/2)
    data['bid_spread'] = data['bid_price1'] - data['bid_price2']
    data['ask_spread'] = data['ask_price1'] - data['ask_price2']
    data['total_volume'] = (data['ask_size1'] + data['ask_size2']) + (data['bid_size1'] + data['bid_size2'])
    data['volume_imbalance'] = abs((data['ask_size1'] + data['ask_size2']) - (data['bid_size1'] + data['bid_size2']))
    #dict for aggregate
    create_feature_dict = {
        'log_return':[realized_volatility],
        'log_return2':[realized_volatility],
        'wap_balance':[np.mean],
        'price_spread':[np.mean],
        'bid_spread':[np.mean],
        'ask_spread':[np.mean],
        'volume_imbalance':[np.mean],
        'total_volume':[np.mean],
        'wap1':[np.mean],
        'wap2':[np.mean],
            }

    #####groupby / all seconds
    df_feature = pd.DataFrame(data.groupby(['minute_id'],sort=False).agg(create_feature_dict)).reset_index()
    
    df_feature.columns = ['_'.join(col) for col in df_feature.columns] #time_id is changed to time_id_
    df_feature["stock_id"]=stock_id
    df_feature["time_id"] = df_feature["minute_id_"].str.split("-").str[0]

    return df_feature
res=preprocessor_book(1)

C:\Users\gaomi\AppData\Local\Temp\ipykernel_31992\3037897417.py:37: FutureWarning: The provided callable <function mean at 0x000002724F774400> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df_feature = pd.DataFrame(data.groupby(['minute_id'],sort=False).agg(create_feature_dict)).reset_index()


,minute_id_,log_return_realized_volatility,log_return2_realized_volatility,wap_balance_mean,price_spread_mean,bid_spread_mean,ask_spread_mean,volume_imbalance_mean,total_volume_mean,wap1_mean,wap2_mean,stock_id,time_id
0,5-0,0.001894,0.002542,0.000326,0.000706,0.000105,-0.000128,164.303571,337.910714,1.002035,1.002030,1,5
1,5-1,0.002064,0.002507,0.000312,0.000676,0.000178,-0.000109,103.438596,244.877193,1.002549,1.002399,1,5
2,5-2,0.002186,0.002464,0.000272,0.000760,0.000125,-0.000086,284.084746,433.847458,1.002817,1.002862,1,5
3,5-3,0.002149,0.002864,0.000236,0.000706,0.000122,-0.000117,127.288136,276.406780,1.005106,1.005145,1,5
4,5-4,0.002286,0.002497,0.000234,0.000679,0.000094,-0.000100,137.050000,322.783333,1.005366,1.005390,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
38295,32767-5,0.000668,0.000546,0.000139,0.000389,0.000110,-0.000106,142.178571,720.750000,1.000580,1.000479,1,32767
38296,32767-6,0.000364,0.000476,0.000050,0.000194,0.000139,-0.000103,156.074074,692.370370,1.000674,1.000642,1,32767
38297,32767-7,0.000322,0.000410,0.000122,0.000346,0.000109,-0.000102,115.366667,861.166667,1.000573,1.000490,1,32767
38298,32767-8,0.000677,0.000902,0.000101,0.000285,0.000106,-0.000141,341.325000,710.525000,0.999953,0.999922,1,32767


In [36]:
res["target"]=res.groupby("time_id")["log_return_realized_volatility"].shift(-1)
res.isna().sum()

minute_id_                            0
log_return_realized_volatility        0
log_return2_realized_volatility       0
wap_balance_mean                      0
price_spread_mean                     0
bid_spread_mean                       0
ask_spread_mean                       0
volume_imbalance_mean                 0
total_volume_mean                     0
wap1_mean                             0
wap2_mean                             0
stock_id                              0
time_id                               0
target                             3830
dtype: int64

In [18]:
dir_path=f"individual_book_train/stock_{stock_id}.csv"
data=pd.read_csv(dir_path)
data['interval_id'] = data['seconds_in_bucket'] // 60

data['minute_id'] = data['time_id'].astype(str) + '-' + data['interval_id'].astype(str)
data = data.drop('interval_id', axis=1)
data

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2,stock_id,minute_id
0,5,0,1.000754,1.001542,1.000689,1.001607,1,25,25,100,1,5-0
1,5,1,1.000754,1.001673,1.000689,1.001739,26,60,25,100,1,5-0
2,5,2,1.000754,1.001411,1.000623,1.001476,1,25,25,125,1,5-0
3,5,3,1.000754,1.001542,1.000689,1.001607,125,25,126,36,1,5-0
4,5,4,1.000754,1.001476,1.000623,1.001542,100,100,25,25,1,5-0
...,...,...,...,...,...,...,...,...,...,...,...,...
1507527,32767,588,0.998911,0.999109,0.998812,0.999208,126,42,101,100,1,32767-9
1507528,32767,589,0.998911,0.999109,0.998812,0.999208,126,126,101,200,1,32767-9
1507529,32767,591,0.998911,0.999109,0.998812,0.999208,126,226,101,200,1,32767-9
1507530,32767,592,0.998911,0.999109,0.998812,0.999208,226,225,101,100,1,32767-9


In [ ]:
#calculate return etc
data['wap1'] =(data['bid_price1'] * data['ask_size1'] + data['ask_price1'] * data['bid_size1'])/(data['bid_size1'] + data['ask_size1'])
data['log_return'] = data.groupby('time_id')['wap1'].transform(log_return)
data['wap2'] = calc_wap2(data)
data['log_return2'] = data.groupby('time_id')['wap2'].transform(log_return)

data['wap_balance'] = abs(data['wap1'] - data['wap2'])

data['price_spread'] = (data['ask_price1'] - data['bid_price1']) / ((data['ask_price1'] + data['bid_price1'])/2)
data['bid_spread'] = data['bid_price1'] - data['bid_price2']
data['ask_spread'] = data['ask_price1'] - data['ask_price2']
data['total_volume'] = (data['ask_size1'] + data['ask_size2']) + (data['bid_size1'] + data['bid_size2'])
data['volume_imbalance'] = abs((data['ask_size1'] + data['ask_size2']) - (data['bid_size1'] + data['bid_size2']))